# Assignment 3 in System Genetics - Gil Sasson & Tomer Schweid

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import GEOparse
from sklearn.feature_selection import f_regression
from statsmodels.stats.multitest import fdrcorrection
from itertools import chain

# Part A: eQTL analysis

## Preprocessing

In [3]:
directory_path = "/Users/gilsasson/Library/CloudStorage/OneDrive-mail.tau.ac.il/ביולוגיה ומדעי המוח/שנה ג/סמסטר ב/גנטיקה בעידן הרפואה האישית/Assignments/Assignment 3/"

In [18]:
# Download the data
gse_myeloid = GEOparse.get_GEO(geo='GSE18067', destdir=directory_path)
gse_liver = GEOparse.get_GEO(geo='GSE17522', destdir=directory_path)

06-Aug-2024 22:06:16 DEBUG utils - Directory /Users/gilsasson/Library/CloudStorage/OneDrive-mail.tau.ac.il/ביולוגיה ומדעי המוח/שנה ג/סמסטר ב/גנטיקה בעידן הרפואה האישית/Assignments/Assignment 3/ already exists. Skipping.
06-Aug-2024 22:06:16 INFO GEOparse - File already exist: using local version.
06-Aug-2024 22:06:16 INFO GEOparse - Parsing /Users/gilsasson/Library/CloudStorage/OneDrive-mail.tau.ac.il/ביולוגיה ומדעי המוח/שנה ג/סמסטר ב/גנטיקה בעידן הרפואה האישית/Assignments/Assignment 3/GSE18067_family.soft.gz: 
06-Aug-2024 22:06:16 DEBUG GEOparse - DATABASE: GeoMiame
06-Aug-2024 22:06:16 DEBUG GEOparse - SERIES: GSE18067
06-Aug-2024 22:06:16 DEBUG GEOparse - PLATFORM: GPL6238
06-Aug-2024 22:06:16 DEBUG GEOparse - SAMPLE: GSM451699
06-Aug-2024 22:06:16 DEBUG GEOparse - SAMPLE: GSM451700
06-Aug-2024 22:06:16 DEBUG GEOparse - SAMPLE: GSM451701
06-Aug-2024 22:06:16 DEBUG GEOparse - SAMPLE: GSM451702
06-Aug-2024 22:06:16 DEBUG GEOparse - SAMPLE: GSM451703
06-Aug-2024 22:06:16 DEBUG GEOparse

In [20]:
# Get the myeloid metadata
metadata_myeloid = pd.read_csv(directory_path + "metadata myeloid.txt", sep="\t")
metadata_myeloid.head()

,GSM name,Strain
0,GSM451699,BXD13 Myeloid batch1
1,GSM451700,BXD23 Erythroid batch1
2,GSM451701,BXD40 Erythroid batch1
3,GSM451702,BXD36 Myeloid batch1
4,GSM451703,BXD6 Stem batch1


In [21]:
# Get the liver metadata
metadata_liver = pd.read_csv(directory_path + "metadata liver.txt", sep="\t")
metadata_liver.head()

,GSM name,Strain
0,GSM436705,Liver_C57BL6J_M_B1_rep1
1,GSM436706,Liver_C57BL6J_M_B1_rep2
2,GSM436707,Liver_C57BL6J_F_B1
3,GSM436708,Liver_DBA2J_F_B1
4,GSM436709,Liver_B6D2F1_M_B1


In [36]:
# Get the gene identifiers for the myeloid data
gene_identifiers_df_myeloid = pd.DataFrame(gse_myeloid.gpls[list(gse_myeloid.gpls.keys())[0]].table)[["ID" , "Symbol"]]
gene_identifiers_df_myeloid.head()

,ID,Symbol
0,GI_38090455-S,LOC382362
1,scl0011717.2_283-S,Ampd3
2,scl52892.1.1_103-S,D830016O14Rik
3,scl0012946.2_3-S,Crry
4,scl18939.11_88-S,Ehf


In [46]:
# Get the gene identifiers for the liver data
gene_identifiers_df_liver= pd.DataFrame(gse_liver.gpls[list(gse_liver.gpls.keys())[0]].table)[["ID" , "GENE_NAME"]]
gene_identifiers_df_liver.head()

,ID,GENE_NAME
0,A_51_P100021,human immunodeficiency virus type I enhancer b...
1,A_51_P100034,MIF4G domain containing
2,A_51_P100052,"SLIT and NTRK-like family, member 2"
3,A_51_P100065,ligand of numb-protein X 1
4,A_51_P100084,NaN


In [162]:
# Define useful functions
def parse_gse(gse):
  final_df = pd.DataFrame(columns=["GSM_name", "ID_REF", "VALUE"])
  sample_dfs = []

  # Iterate through each sample
  for gsm_name, gsm in gse.gsms.items():
      sample_data = {
          "GSM_name": gsm_name,
          "ID_REF": gsm.table["ID_REF"],
          "VALUE": gsm.table["VALUE"],
      }
      sample_df = pd.DataFrame(sample_data)
      sample_dfs.append(sample_df)
  final_df = pd.concat(sample_dfs, ignore_index=True)
  return final_df

def merge(df1, df2, left_on, right_on):
    df1 = df1.merge(df2, left_on=left_on, right_on=right_on)
    return df1.drop(columns=[right_on, left_on])

def filter_samples(df, cell_type="Myeloid"):
    # Extract relevant cell types
    df = df[df["Strain"].str.contains(cell_type)]
    # Drop the technical replicates
    df = df[~df["Strain"].str.contains("Technical")]
    df = df[~df["Strain"].str.contains("rep")]
    return df

def pivot_strains(df_merged, index):
    # Pivot the data
    df_pivot = df_merged.pivot_table(columns="Strain", values="VALUE", index=index, aggfunc="mean")
    return df_pivot

def mean_batches(df, cell_type="Myeloid"):
    if cell_type == "Myeloid":
        batch_names = ["a", "b"]
        # Extract strain names from column names
        df.columns = pd.MultiIndex.from_tuples([col.split(' ', 1) for col in df.columns], names=['strain', 'batch'])
        # Calculate the mean across batches for each strain
        mean_df = df.T.groupby(level='strain').mean()
        return mean_df.T
    elif cell_type == "Liver":
        # Mean across sex for each strain
        df.columns = pd.MultiIndex.from_tuples([col.split('_', 2) for col in df.columns],names=['liver','strain','sex'])
        # Calculate the mean across sex for each strain
        mean_df = df.T.groupby(level=['strain']).mean()
        return mean_df.T
    

def mean_strains(df_pivot):
        for strain in df_pivot.columns:
            if strain[-1] == 'a' or strain[-1] == 'A':
                strain_name = strain[:-1]
                if strain_name + 'b' in df_pivot.columns:
                    df_pivot[strain_name] = df_pivot[[strain, strain_name + 'b']].mean(axis=1)
                    df_pivot = df_pivot.drop(columns=[strain, strain_name + 'b'])
                else:
                    df_pivot[strain_name] = df_pivot[strain]
                    df_pivot = df_pivot.drop(columns=[strain])
            elif strain[-1] == 'b' :
                strain_name = strain[:-1]
                if strain_name + 'a' in df_pivot.columns:
                    df_pivot[strain_name] = df_pivot[[strain, strain_name + 'a']].mean(axis=1)
                    df_pivot = df_pivot.drop(columns=[strain, strain_name + 'a'])
        return df_pivot

In [163]:
# Get the  myeloid data
myeloid_df = parse_gse(gse_myeloid)
# Merge the metadata with the data
myeloid_df = merge(myeloid_df, metadata_myeloid, "GSM_name", "GSM name")
# Merge the gene identifiers with the data
myeloid_df = merge(myeloid_df, gene_identifiers_df_myeloid, "ID_REF", "ID")
# Filter the samples to include only the myeloid cells data
myeloid_df = filter_samples(myeloid_df)
# Reshape the data
myeloid_df = pivot_strains(myeloid_df, "Symbol")
# Calculate the mean across batches for each strain
myeloid_df = mean_batches(myeloid_df)
# Merge duplicate strains columns
myeloid_df = mean_strains(myeloid_df)
myeloid_df

strain,BXD1,BXD11,BXD12,BXD13,BXD14,BXD16,BXD18,BXD19,BXD2,BXD21B,...,BXD32B,BXD34B,BXD36,BXD40,BXD42,BXD6,BXD8,BXD9,BXD28,BXD33
Symbol,,,,,,,,,,,,,,,,,,,,,
0610005A07Rik,6.439252,6.418787,6.461313,6.459590,6.392369,6.515128,6.446004,6.392673,6.263298,6.318342,...,6.391703,6.414189,6.339669,6.244116,6.322840,6.480558,6.522814,6.373272,6.529782,6.513522
0610005C13Rik,6.462294,6.556949,6.591845,6.573851,6.482562,6.616155,6.494463,6.553485,6.492469,6.409468,...,6.616821,6.510064,6.530035,6.591437,6.526547,6.454843,6.496264,6.586568,6.604400,6.608433
0610005H09Rik,6.960774,7.096886,7.711693,7.647623,7.651882,7.734397,7.822844,7.631427,6.896540,7.810171,...,8.154312,7.292728,8.094586,8.252535,7.375777,7.562560,7.577809,7.829545,7.516431,7.651062
0610005I04,6.532020,6.594255,6.489756,6.697061,6.497515,6.548089,6.637054,6.616757,6.633698,6.774890,...,6.711600,6.488575,6.650941,6.705175,6.630276,6.831132,6.739256,6.606980,6.953100,6.804694
0610005K03Rik,7.661794,7.659268,7.786213,7.787414,8.152209,8.030581,7.836622,8.056267,7.666384,7.882478,...,7.776629,8.313372,7.908074,7.922678,7.836622,7.928899,7.988844,7.882973,7.781489,7.780241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p,6.541973,6.479159,6.212613,6.386862,6.508521,6.347702,6.493394,6.104731,6.359801,6.420321,...,6.518097,6.369921,6.395560,6.400445,6.347738,6.517618,6.426932,6.535042,6.642090,6.336126
scavenger receptor,7.106817,7.526465,7.402048,7.746785,6.798596,8.034765,8.092333,6.715678,7.059044,7.012611,...,7.850640,7.550843,6.652632,7.252858,6.451716,6.899810,6.699562,7.049678,7.132313,6.960724
sty,8.787945,9.646043,9.472719,8.881412,9.161894,9.242461,8.409188,9.029700,8.523423,8.381539,...,8.721376,7.972074,8.622802,9.116669,9.550628,7.741574,9.063488,8.008797,9.239431,9.241477


In [164]:
# Get the liver data
liver_df = parse_gse(gse_liver)
# Merge the metadata with the data
liver_df = merge(liver_df, metadata_liver, "GSM_name", "GSM name")
# Merge the gene identifiers with the data
liver_df = merge(liver_df, gene_identifiers_df_liver, "ID_REF", "ID")
# Drop technical replicates
liver_df = filter_samples(liver_df, "Liver")
# Reshape the data
liver_df = pivot_strains(liver_df, "GENE_NAME")
# Calculate the mean across sex for each strain
liver_df = mean_batches(liver_df, "Liver")
liver_df

(2545896, 3)
(2086800, 3)
(17917, 100)


strain,B6D2F1,BXD1,BXD11,BXD11TY,BXD12,BXD13,BXD14,BXD15,BXD16,BXD19,...,BXD69,BXD73,BXD77,BXD8,BXD85,BXD86,BXD9,BXD92,C57BL6J,DBA2J
GENE_NAME,,,,,,,,,,,,,,,,,,,,,
"1-acylglycerol-3-phosphate O-acyltransferase 1 (lysophosphatidic acid acyltransferase, alpha)""",1.210833,1.7480,2.243,1.113,1.2085,1.9485,0.9215,1.0310,1.1545,1.5565,...,1.2765,0.7425,1.4360,1.7245,1.3890,1.4395,1.1540,1.3800,1.12200,1.3166
"1-acylglycerol-3-phosphate O-acyltransferase 2 (lysophosphatidic acid acyltransferase, beta)",1.598000,1.7625,1.905,1.981,1.5790,1.9415,1.3830,1.8660,2.0565,2.3185,...,1.9495,1.4490,1.8260,2.0670,1.7725,1.8180,1.2430,1.7840,1.75100,1.8952
1-acylglycerol-3-phosphate O-acyltransferase 3,0.421167,0.7030,0.508,0.040,0.5605,0.3715,0.2740,0.6665,0.4685,0.3900,...,0.4090,0.2680,0.4305,0.6770,0.3760,0.4680,0.5405,0.2110,0.53600,0.6947
"1-acylglycerol-3-phosphate O-acyltransferase 4 (lysophosphatidic acid acyltransferase, delta)""",-2.789833,-2.7585,-2.945,-2.986,-2.7850,-3.0230,-2.6615,-2.7080,-3.2560,-3.1915,...,-2.9405,-2.8110,-3.0055,-2.9290,-3.1155,-2.9195,-2.7315,-2.9565,-2.92500,-2.6595
"1-acylglycerol-3-phosphate O-acyltransferase 5 (lysophosphatidic acid acyltransferase, epsilon)",-0.886667,-0.6520,-1.477,-1.480,-0.5300,-1.1370,-0.8910,-0.2990,-1.7375,-1.6430,...,-1.6545,-1.7400,-0.6750,-0.4900,-1.8610,-0.6505,-0.5690,-0.6770,-0.61000,-1.3286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zona pellucida like domain containing 1,-0.206167,-0.1685,-0.153,-0.094,-0.1950,-1.0575,-0.0735,-0.2225,-0.0810,-0.1225,...,0.0140,-0.1100,-0.3550,0.0940,-0.1725,-0.3360,0.0930,-0.1780,-0.03725,-0.0173
zonadhesin,0.279500,0.2415,0.438,0.350,0.2925,0.2875,0.3600,0.2525,0.3925,0.2340,...,0.0630,0.3440,0.0840,0.2090,0.2685,0.1960,0.7185,0.0780,0.40250,0.3905
zyg-11 homolog A (C. elegans),0.458167,0.4280,0.728,0.773,0.3185,0.2415,0.6095,0.0635,0.6660,0.4840,...,0.5350,0.2815,0.7390,0.6840,0.3355,0.6170,0.5690,0.6635,0.44575,0.2731


In [165]:
# Get the genotype data
genotype_df = pd.read_csv(directory_path + "genotypes.txt")
genotype_df.head()

,0,Locus,Chr_Build37,Build37_position,BXD1,BXD2,BXD5,BXD6,BXD8,BXD9,...,BXD94,BXD95,BXD96,BXD97,BXD98,BXD99,BXD100,BXD101,BXD102,BXD103
0,1,rs6269442,1,3482276,0,0,2,2,2,0,...,2,2,0,0,0,0,0,U,U,U
1,2,rs6365999,1,4811063,0,0,2,2,2,0,...,2,2,0,0,0,0,0,U,U,U
2,3,rs6376963,1,5008090,0,0,2,2,2,0,...,2,2,0,0,0,0,0,U,U,U
3,4,rs3677817,1,5176059,0,0,2,2,2,0,...,2,2,0,0,0,0,0,U,U,U
4,5,rs8236463,1,5579194,0,0,2,2,2,0,...,2,2,0,0,0,0,0,U,U,U


In [166]:
# Filter the relevant strains to include only those present in myeloid_df
relevant_strains = genotype_df.columns[4:]
# Filter the relevant strains to include only those present in myeloid_df
filtered_strains_myeloid = [col for col in relevant_strains if col in myeloid_df.columns]
# Select the filtered columns from myeloid_df
myeloid_df = myeloid_df.loc[:, filtered_strains_myeloid]
# Get only the strains present in myeolid_df from the genotype data
representative_snps_myeloid = genotype_df[filtered_strains_myeloid]
# Now look for snps that have exactly same values for all the strains
duplicates = representative_snps_myeloid[representative_snps_myeloid.duplicated()]
print(f"The number of duplicate SNPs is: {len(duplicates)}")
# Remove the duplicate SNPs
representative_snps_liver = representative_snps_myeloid.drop_duplicates()

strain,BXD1,BXD2,BXD6,BXD8,BXD9,BXD11,BXD12,BXD13,BXD14,BXD16,...,BXD19,BXD23,BXD27,BXD28,BXD29,BXD31,BXD33,BXD36,BXD40,BXD42
Symbol,,,,,,,,,,,,,,,,,,,,,
0610005A07Rik,6.439252,6.263298,6.480558,6.522814,6.373272,6.418787,6.461313,6.459590,6.392369,6.515128,...,6.392673,6.343721,6.486555,6.529782,6.409871,6.494575,6.513522,6.339669,6.244116,6.322840
0610005C13Rik,6.462294,6.492469,6.454843,6.496264,6.586568,6.556949,6.591845,6.573851,6.482562,6.616155,...,6.553485,6.531997,6.587381,6.604400,6.592969,6.446492,6.608433,6.530035,6.591437,6.526547
0610005H09Rik,6.960774,6.896540,7.562560,7.577809,7.829545,7.096886,7.711693,7.647623,7.651882,7.734397,...,7.631427,7.658030,7.765902,7.516431,7.733955,7.371241,7.651062,8.094586,8.252535,7.375777
0610005I04,6.532020,6.633698,6.831132,6.739256,6.606980,6.594255,6.489756,6.697061,6.497515,6.548089,...,6.616757,6.640003,6.907307,6.953100,6.727989,6.721578,6.804694,6.650941,6.705175,6.630276
0610005K03Rik,7.661794,7.666384,7.928899,7.988844,7.882973,7.659268,7.786213,7.787414,8.152209,8.030581,...,8.056267,7.920657,7.900948,7.781489,7.869633,7.581931,7.780241,7.908074,7.922678,7.836622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p,6.541973,6.359801,6.517618,6.426932,6.535042,6.479159,6.212613,6.386862,6.508521,6.347702,...,6.104731,6.359801,6.466790,6.642090,6.369591,6.415152,6.336126,6.395560,6.400445,6.347738
scavenger receptor,7.106817,7.059044,6.899810,6.699562,7.049678,7.526465,7.402048,7.746785,6.798596,8.034765,...,6.715678,7.140271,6.874077,7.132313,7.834666,7.702109,6.960724,6.652632,7.252858,6.451716
sty,8.787945,8.523423,7.741574,9.063488,8.008797,9.646043,9.472719,8.881412,9.161894,9.242461,...,9.029700,9.350928,9.545547,9.239431,9.492746,10.106197,9.241477,8.622802,9.116669,9.550628


In [167]:
# Now we have the data in the correct format
# We can start filtering the data
# First, we will filter out the genes with no gene identifier
myeloid_df_filtered = myeloid_df.dropna()
# Then, we will filter out the genes with low variance
row_variance = myeloid_df_filtered.var(axis=1)
average_variance = row_variance.mean()
print(f"The average variance is: {round(average_variance, 3)}")
myeloid_df_filtered = myeloid_df[row_variance > average_variance/10]
# Now we will filter out the genes with low expression
row_max = myeloid_df_filtered.max(axis=1)
average_max = row_max.mean()
print(f"The average maximum expression value is: {round(average_max, 3)}")
myeloid_df_filtered = myeloid_df_filtered[row_max > average_max-1.2]
print(f"The number of genes before filtering is: {len(myeloid_df)} \nThe number of genes after filtering is: {len(myeloid_df_filtered)}")
myeloid_df_filtered

The average variance is: 0.044
The average maximum expression value is: 7.661
The number of duplicate SNPs is: 3076


In [168]:
# Filter the relevant strains to include only those present in myeloid_df
relevant_strains = genotype_df.columns[4:]
# Filter the relevant strains to include only those present in myeloid_df
filtered_strains_liver = [col for col in relevant_strains if col in liver_df.columns]
# Select the filtered columns from liver_df
liver_df = liver_df.loc[:, filtered_strains_liver]
# Get only the strains present in liver_df from the genotype data
# Filter the genotype data to include only the relevant strains
representative_snps_liver = genotype_df[filtered_strains_liver]
# Now look for snps that have exactly same values for all the strains
duplicates = representative_snps_liver[representative_snps_liver.duplicated()]
print(f"The number of duplicate SNPs is: {len(duplicates)}")
# Remove the duplicate SNPs
representative_snps_liver = representative_snps_liver.drop_duplicates()

strain,BXD1,BXD2,BXD5,BXD6,BXD8,BXD9,BXD11,BXD12,BXD13,BXD14,...,BXD48,BXD51,BXD60,BXD62,BXD69,BXD73,BXD77,BXD85,BXD86,BXD92
GENE_NAME,,,,,,,,,,,,,,,,,,,,,
"1-acylglycerol-3-phosphate O-acyltransferase 1 (lysophosphatidic acid acyltransferase, alpha)""",1.7480,1.1440,0.8800,1.2030,1.7245,1.1540,2.243,1.2085,1.9485,0.9215,...,0.8665,0.9135,1.0820,0.9410,1.2765,0.7425,1.4360,1.3890,1.4395,1.3800
"1-acylglycerol-3-phosphate O-acyltransferase 2 (lysophosphatidic acid acyltransferase, beta)",1.7625,1.8615,1.7190,1.6200,2.0670,1.2430,1.905,1.5790,1.9415,1.3830,...,2.1070,1.9635,1.5355,1.7440,1.9495,1.4490,1.8260,1.7725,1.8180,1.7840
1-acylglycerol-3-phosphate O-acyltransferase 3,0.7030,0.2645,0.6325,0.3815,0.6770,0.5405,0.508,0.5605,0.3715,0.2740,...,0.2860,0.3145,0.5700,0.5715,0.4090,0.2680,0.4305,0.3760,0.4680,0.2110
"1-acylglycerol-3-phosphate O-acyltransferase 4 (lysophosphatidic acid acyltransferase, delta)""",-2.7585,-2.9470,-2.5170,-2.5420,-2.9290,-2.7315,-2.945,-2.7850,-3.0230,-2.6615,...,-3.0545,-2.8490,-2.4800,-2.9070,-2.9405,-2.8110,-3.0055,-3.1155,-2.9195,-2.9565
"1-acylglycerol-3-phosphate O-acyltransferase 5 (lysophosphatidic acid acyltransferase, epsilon)",-0.6520,-0.1725,-1.1295,-0.4600,-0.4900,-0.5690,-1.477,-0.5300,-1.1370,-0.8910,...,-0.5345,-0.7830,-1.2535,-0.7695,-1.6545,-1.7400,-0.6750,-1.8610,-0.6505,-0.6770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zona pellucida like domain containing 1,-0.1685,-0.1325,-0.4075,-0.3370,0.0940,0.0930,-0.153,-0.1950,-1.0575,-0.0735,...,-0.1560,-0.1410,-0.0850,-0.1955,0.0140,-0.1100,-0.3550,-0.1725,-0.3360,-0.1780
zonadhesin,0.2415,0.3035,0.2125,0.3725,0.2090,0.7185,0.438,0.2925,0.2875,0.3600,...,0.3075,0.2205,0.2730,0.2175,0.0630,0.3440,0.0840,0.2685,0.1960,0.0780
zyg-11 homolog A (C. elegans),0.4280,0.8640,0.2355,0.3880,0.6840,0.5690,0.728,0.3185,0.2415,0.6095,...,0.4615,0.5400,0.3875,0.5035,0.5350,0.2815,0.7390,0.3355,0.6170,0.6635


In [169]:
# Now we have the data in the correct format
# We can start filtering the data
# first, we will filter out the genes with no gene identifier
liver_df_filtered = liver_df.dropna()
# Then, we will filter out the genes with low variance
row_variance = liver_df_filtered.var(axis=1)
average_variance = row_variance.mean()
print(f"The average variance is: {round(average_variance, 3)}")
liver_df_filtered = liver_df[row_variance > average_variance/10]
# Now we will filter out the genes with low expression
row_max = liver_df_filtered.max(axis=1)
average_max = row_max.mean()
print(f"The average maximum expression value is: {round(average_max, 3)}")
liver_df_filtered = liver_df_filtered[row_max > average_max-2]
print(f"The number of genes before filtering is: {len(liver_df)} \nThe number of genes after filtering is: {len(liver_df_filtered)}")
liver_df_filtered

The average variance is: 0.055
The average maximum expression value is: 0.275
The number of duplicate SNPs is: 2424


In [173]:
def association_test(df, snps):
    eQTLs = []
    for gene in df.index:
        # Run the regression
        results = f_regression(snps.T, df.loc[gene].T)
        # Get the p-values
        p_values = results[1]
        eQTLs.append({"gene": gene, "p_values": p_values})
    eQTLs_df = pd.DataFrame(eQTLs)
    return eQTLs_df

def correct_results(df1, df2):
    # Merge the p-values columns for analysis
    p_merged = pd.concat([df1['p_values'], df2['p_values']], axis=0)
    # Correct the p-values
    p_values = list(chain.from_iterable(p_merged))
    corrected_p_values = fdrcorrection(p_values)[1]
    # Initialize new columns
    df1["corrected_p_values"] = [None] * len(df1)
    df1["significant_snps"] = [None] * len(df1)
    df2["corrected_p_values"] = [None] * len(df2)
    df2["significant_snps"] = [None] * len(df2)
    for df in [df1, df2]:
        n_snps = len(df.loc[0, "p_values"])
        for i in range(len(df)):
            start = i * n_snps
            end = (i + 1) * n_snps
            # Assign corrected p-values for each gene
            df.at[i, "corrected_p_values"] = corrected_p_values[start:end]
            # Identify significant SNPs for each gene
            df.at[i, "significant_snps"] = {genotype_df.loc[i, 'Locus']: j for i, j in enumerate(corrected_p_values[start:end]) if j < 0.05}
    # drop the genes with no significant SNPs
    eQTLs_df1= df1[df1["significant_snps"].map(lambda d: len(d)) > 0].reset_index(drop=True)
    eQTLs_df2 = df2[df2["significant_snps"].map(lambda d: len(d)) > 0].reset_index(drop=True)
    return eQTLs_df1, eQTLs_df2

def eqtl_analysis(df1, df2, snps1, snps2):
    eQTLs1 = association_test(df1, snps1)
    eQTLs2 = association_test(df2, snps2)
    eQTLs_corrected1, eQTLs_corrected2 = correct_results(eQTLs1, eQTLs2)
    return eQTLs_corrected1, eQTLs_corrected2

In [175]:
# Run the eQTL analysis
eQTLs_myeloid, eQTLs_liver = eqtl_analysis(myeloid_df_filtered, liver_df_filtered, representative_snps_myeloid, representative_snps_liver)
eQTLs_myeloid.head()

,gene,p_values,corrected_p_values,significant_snps
0,0610009K11Rik,"[0.7873410843132229, 0.7873410843132229, 0.787...","[0.9986926025347768, 0.9986926025347768, 0.998...","{'rs13478002': 0.01344690210426767, 'rs4224864..."
1,0710001D07Rik,"[0.5113191855604085, 0.5113191855604085, 0.511...","[0.9965966668568766, 0.9965966668568766, 0.996...","{'rs13482934': 0.00026800301316723505, 'rs3693..."
2,1110001E17Rik,"[0.42799892033109144, 0.42799892033109144, 0.4...","[0.9957370957867651, 0.9957370957867651, 0.995...","{'rs13460430': 0.03851564429008389, 'rs3685305..."
3,1110003E01Rik,"[0.9394831525677712, 0.9394831525677712, 0.939...","[0.9996599129712701, 0.9996599129712701, 0.999...","{'rs3657916': 0.0122725734536789, 'rs13478293'..."
4,1110005F07Rik,"[0.8871530382716325, 0.8871530382716325, 0.887...","[0.9992968818911117, 0.9992968818911117, 0.999...","{'rs13478002': 0.002240700027782331, 'rs422486..."


In [176]:
eQTLs_liver.head()

,gene,p_values,corrected_p_values,significant_snps
0,A kinase (PRKA) anchor protein 3,"[0.6830489507099937, 0.6850456551690991, 0.960...","[0.9989869813942279, 0.9989869813942279, 0.998...","{'rs6290401': 0.01344690210426767, 'rs13477462..."
1,ArfGAP with dual PH domains 2,"[0.6069728753745729, 0.7059930847210809, 0.682...","[0.9958406034730389, 0.9958406034730389, 0.995...","{'rs13475988': 0.00026800301316723505, 'rs1347..."
2,"C-type lectin domain family 2, member h""","[0.8348079984933076, 0.9576996905427395, 0.715...","[0.999985044995347, 0.999985044995347, 0.99998...","{'rs6218880': 0.03851564429008389, 'rs13478667..."
3,"C-type lectin domain family 2, member i""","[0.06144517396454861, 0.06813147768347926, 0.1...","[0.03851564429008389, 0.03851564429008389, 0.7...","{'rs6269442': 0.03851564429008389, 'rs6365999'..."
4,"CCR4-NOT transcription complex, subunit 7""","[0.7959902314177908, 0.6422387765379605, 0.761...","[0.9983927069876475, 0.9983927069876475, 0.994...","{'rs3660779': 0.0122725734536789, 'rs13476533'..."


In [181]:
eQTLs_myeloid["gene"]

0           0610009K11Rik
1           0710001D07Rik
2           1110001E17Rik
3           1110003E01Rik
4           1110005F07Rik
              ...        
585                Zfp297
586                 Zfp39
587                 Zfp68
588                 Zzef1
589    scavenger receptor
Name: gene, Length: 590, dtype: object

In [ ]:
eQTLs = []
for eqtl in eQTLs_myeloid["significant_snps"]:
    eQTLs.extend(eqtl.keys())
# count how many times each SNP appears
eQTLs = pd.Series(eQTLs)
eQTLs = eQTLs.value_counts()
figure = plt.figure(figsize=(15, 10))
sns.barplot(x=eQTLs.index, y=eQTLs.values)
plt.xlabel("Number of eQTLs")
plt.ylabel("Frequency")
plt.title("Frequency of eQTLs")
plt.xticks(rotation=45, fontsize=12)
plt.xlim(0, 50)
plt.show()